In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

In [ ]:
df = sqlContext.read.load('inputFile.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')
#df.count()

In [ ]:
df.show()

In [ ]:
#open picked model
serialized_model = open("pickled_model.sav", "rb")
model = pickle.load(serialized_model)
serialized_model.close()
#broadcast model to spark executors using spark context(sc)
sc.broadcast(model)
#update prediction method
def predictor(s_l, s_w, p_l, p_w):
    #call predict method for model
    return model.predict([s_l, s_w, p_l, p_w])
#register python method as spark UDF and call over dataframe
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
udf_predictor = udf(predictor, FloatType())
#apply the udf to dataframe
df_prediction = df.withColumn("prediction",
                                udf_predictor(df.a    
                                              ,df.b
                                              , df.c
                                              , df.d))
                                            

In [ ]:
df_prediction.coalesce(1).write.format('com.databricks.spark.csv').option('header', 'true').save('predictions.csv')